In [ ]:
!pip install -U spacy
!pip install spacy_transformers

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pickle

In [ ]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [ ]:
data = trim_entity_spans(convert_dataturks_to_spacy("/content/Entity Recognition in Resumes.json"))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

In [ ]:
da

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
# file_path = '/content/output'

# # Use the !rm command to remove the file
# !rm -r "{file_path}"

In [ ]:
db = DocBin()
i = 0
for text, annot in tqdm(data):
    try:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot['entities']:
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            if span is None:
                print('Skipping entity')
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
        i += 1
        print('loaded', i)
    except Exception as e:
        print(f"Error processing example: {e}")

# Print the total number of resumes processed
print(f'Total resumes processed: {i}')

In [ ]:
db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.1 )

In [ ]:
db.to_disk('train_data.spacy')
db.to_disk('test_data.spacy')

In [ ]:
!python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

2024-02-01 05:14:21.075658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 05:14:21.075713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 05:14:21.076924: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 05:14:22.096575: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2024-02-01 05:17:24.834770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 05:17:24.834814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 05:17:24.836005: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 05:17:25.943713: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint

In [ ]:
!zip -r /content/main_file.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-best/ (stored 0%)
  adding: content/output/model-best/meta.json (deflated 68%)
  adding: content/output/model-best/config.cfg (deflated 61%)
  adding: content/output/model-best/ner/ (stored 0%)
  adding: content/output/model-best/ner/model (deflated 8%)
  adding: content/output/model-best/ner/cfg (deflated 33%)
  adding: content/output/model-best/ner/moves (deflated 73%)
  adding: content/output/model-best/vocab/ (stored 0%)
  adding: content/output/model-best/vocab/vectors (deflated 45%)
  adding: content/output/model-best/vocab/key2row (stored 0%)
  adding: content/output/model-best/vocab/lookups.bin (stored 0%)
  adding: content/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-best/vocab/strings.json (deflated 76%)
  adding: content/output/model-best/tokenizer (deflated 81%)
  adding: content/output/model-best/transformer/ (stored 0%)
  adding: content/output/model-best/transformer/mod

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        # converting each page into text
        text += page.get_text()
    return text

In [ ]:
pdf_path='/content/AWAIS ZUBAIR.pdf'
resume_text = extract_text_from_pdf(pdf_path)

In [ ]:
text = resume_text.strip()

In [ ]:
text = ' '.join(text.split())

In [ ]:
nlp = spacy.load(r"./output/model-best")

In [ ]:
!python -m spacy validate

2024-02-01 08:08:42.591744: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 08:08:42.591791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 08:08:42.595872: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 08:08:44.770891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation: /usr/local/lib/python3.10/dist-packages/spacy

NAME             SPACY            VERSION                      

In [ ]:
custom_entities

[('Maryam Zubair', 'Name'),
 ('Fremont', 'Location'),
 ('mariyam051@gmail', 'Email Address'),
 ('Master of Science (M.S.), Computer Science, GPA: 3.7', 'Degree'),
 ('San Francisco Bay University', 'College Name'),
 ('Fremont', 'Location'),
 ('Bachelor of Science (B.S.), Electrical Telecommunications Engineering',
  'Degree'),
 ('COMSATS Institute of Information Technology', 'College Name'),
 ('2016', 'Graduation Year'),
 ('Languages: Python, JavaScript, SQL \nFrameworks: Express, Flask, LangChain \nLibraries: scikit-learn, Pandas, NumPy \nMachine Learning & Artificial Intelligence: Experienced training machine learning models, developing \nAI-powered applications utilizing OpenAI APIs, and designing efficient pipelines. \nData Analysis and Visualization: Proficient in creating dashboards using Power BI  \nPROJECTS \nAI-powered Chatbot (Flask, OpenAI API, LangChain, ChromaDB) \n• \nDesigned fully functional multilingual chat application responsible to assist prospective students \nand s